In [31]:
import random
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
import pandas as pd
#import imageio
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data

import matplotlib.pyplot as plt
%matplotlib inline
import math

import torchvision
import torchvision.transforms as transforms
import optuna#载入optuna优化包

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])



# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)

Files already downloaded and verified


In [32]:
##optuna######
###耗费时间的过程。。。。。。。。。。。。。。。
# Define a basic convolutional layer
# 定义带参数的 GELU 激活函数层
class GELUB(nn.Module): 
    def __init__(self, seqFlag, trial):
        super().__init__()
        self.sigma = trial.suggest_float(f'$\sigma_{seqFlag}$', 0.0, 10.0)

    def forward(self, input):
        x = input * (1 + torch.erf(input / math.sqrt(2) / self.sigma)) / 2
        return x

def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

# 定义ResNet模型
class ResNet(nn.Module):
    def __init__(self, trial, num_classes=10):
        super(ResNet, self).__init__()
        self.conv1 = conv3x3(3, 16)
        self.bn1 = nn.BatchNorm2d(16)
        self.custom1 = GELUB(1, trial)  # 自定义激活函数层

        # Layer 1
        self.layer1_conv1 = conv3x3(16, 16)
        self.layer1_bn1 = nn.BatchNorm2d(16)
        self.layer1_conv2 = conv3x3(16, 16)
        self.layer1_bn2 = nn.BatchNorm2d(16)
        self.layer1_extra_conv1 = conv3x3(16, 16)
        self.layer1_extra_bn1 = nn.BatchNorm2d(16)
        self.layer1_extra_conv2 = conv3x3(16, 16)
        self.layer1_extra_bn2 = nn.BatchNorm2d(16)
        self.gelub11 = GELUB(2, trial)
        self.gelub12 = GELUB(3, trial)
        self.gelub13 = GELUB(4, trial)
        self.gelub14 = GELUB(5, trial)

        # Layer 2
        self.layer2_conv1 = conv3x3(16, 32, stride=2)
        self.layer2_bn1 = nn.BatchNorm2d(32)
        self.layer2_conv2 = conv3x3(32, 32)
        self.layer2_bn2 = nn.BatchNorm2d(32)
        self.layer2_extra_conv1 = conv3x3(16, 32, stride=2)  # 调整residual的通道数
        self.layer2_extra_bn1 = nn.BatchNorm2d(32)
        self.layer2_extra_conv2 = conv3x3(32, 32)
        self.layer2_extra_bn2 = nn.BatchNorm2d(32)
        self.gelub21 = GELUB(6, trial)
        self.gelub22 = GELUB(7, trial)
        self.gelub23 = GELUB(8, trial)
        self.gelub24 = GELUB(9, trial)
        self.layer2_downsample = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=1, stride=2, bias=False),
            nn.BatchNorm2d(32)
        )

        # Layer 3
        self.layer3_conv1 = conv3x3(32, 64, stride=2)
        self.layer3_bn1 = nn.BatchNorm2d(64)
        self.layer3_conv2 = conv3x3(64, 64)
        self.layer3_bn2 = nn.BatchNorm2d(64)
        self.layer3_extra_conv1 = conv3x3(32, 64, stride=2)  # 调整residual的通道数
        self.layer3_extra_bn1 = nn.BatchNorm2d(64)
        self.layer3_extra_conv2 = conv3x3(64, 64)
        self.layer3_extra_bn2 = nn.BatchNorm2d(64)
        self.gelub31 = GELUB(10, trial)
        self.gelub32 = GELUB(11, trial)
        self.gelub33 = GELUB(12, trial)
        self.gelub34 = GELUB(13, trial)

        self.layer3_downsample = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=1, stride=2, bias=False),
            nn.BatchNorm2d(64)
        )

        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        # Layer 0
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.custom1(out)

        # Layer 1
        residual = out
        out = self.layer1_conv1(out)
        out = self.layer1_bn1(out)
        out = self.gelub11(out)
        out = self.layer1_conv2(out)
        out = self.layer1_bn2(out)
        out += residual
        out = self.gelub12(out)
        
        residual = out
        out = self.layer1_extra_conv1(out)
        out = self.layer1_extra_bn1(out)
        out = self.gelub13(out)
        out = self.layer1_extra_conv2(out)
        out = self.layer1_extra_bn2(out)
        out += residual
        out = self.gelub14(out)

        # Layer 2
        residual = out
        out = self.layer2_conv1(out)
        out = self.layer2_bn1(out)
        out = self.gelub21(out)
        out = self.layer2_conv2(out)
        out = self.layer2_bn2(out)
        out = self.gelub22(out)
        #if out.size(1) != residual.size(1):
        #   residual = self.layer2_extra_conv1(residual)
        residual = self.layer2_downsample(residual)
        out += residual
        out = self.gelub22(out)

        residual = out
        out = self.layer2_extra_conv2(out)
        out = self.layer2_extra_bn2(out)
        out = self.gelub23(out)
        out = self.layer2_extra_conv2(out)
        out = self.layer2_extra_bn2(out)
        out += residual
        out = self.gelub24(out)

        # Layer 3
        residual = out
        out = self.layer3_conv1(out)
        out = self.layer3_bn1(out)
        out = self.gelub31(out)
        out = self.layer3_conv2(out)
        out = self.layer3_bn2(out)
        residual = self.layer3_downsample(residual)
        out += residual
        out = self.gelub32(out)
        
        residual = out
        out = self.layer3_extra_conv2(out)
        out = self.layer3_extra_bn2(out)
        out = self.gelub33(out)
        out = self.layer3_extra_conv2(out)
        out = self.layer3_extra_bn2(out)
        out += residual
        out = self.gelub34(out)

        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [33]:
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# 定义目标函数

def objective(trial):
    # 模型实例化
    model = ResNet(trial)
    model.to(device)

    # 损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # 保持 Adam 优化器
    num_epochs = 80
    curr_lr = 0.001
    # 初始化正确的标签个数
    PGELU_val_correct = 0

    # 训练模型
    for epoch in range(num_epochs):
        model.train()
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Print training loss
            total_step = len(train_loader)
            if (i+1) % 100 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                      .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

        # Update learning rate every 20 epochs
        if (epoch+1) % 20 == 0:
            curr_lr /= 3
            update_lr(optimizer, curr_lr)

        # 在每个 epoch 结束后打印验证集的准确率
        with torch.no_grad():
            correct = 0
            total = 0
            for images, labels in test_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            epoch_acc = correct / total * 100
            print("Epoch:{}/{} \t AVERAGE VA:{:.2f} %".format(epoch + 1, num_epochs, epoch_acc))

        PGELU_val_correct += correct  # 更新正确的标签个数

    PGELU_valid_acc = PGELU_val_correct / len(test_loader.sampler) * 100
    print("Epoch:{}/{} \t AVERAGE VA:{:.2f} %".format(epoch + 1, num_epochs, PGELU_valid_acc))

    return PGELU_valid_acc
    
study = optuna.create_study(direction='maximize')

# 运行优化过程
study.optimize(objective, n_trials=10)

# 打印最佳参数和目标值
print('Best trial:')
print('  Value: ', study.best_trial.value)
print('  Params: ')
for key, value in study.best_trial.params.items():
    print('    {}: {}'.format(key, value))

[I 2024-04-18 22:46:11,709] A new study created in memory with name: no-name-27a49e0e-8cbe-4704-9bea-b0da58e1d7a8


Epoch [1/80], Step [100/500], Loss: 1.7572
Epoch [1/80], Step [200/500], Loss: 1.3904
Epoch [1/80], Step [300/500], Loss: 1.5464
Epoch [1/80], Step [400/500], Loss: 1.4364
Epoch [1/80], Step [500/500], Loss: 1.3711
Epoch:1/80 	 AVERAGE VA:52.95 %
Epoch [2/80], Step [100/500], Loss: 1.1791
Epoch [2/80], Step [200/500], Loss: 1.1709
Epoch [2/80], Step [300/500], Loss: 1.1572
Epoch [2/80], Step [400/500], Loss: 0.9762


[W 2024-04-18 22:48:25,966] Trial 0 failed with parameters: {'$\\sigma_1$': 1.9163312753791328, '$\\sigma_2$': 6.614917436395135, '$\\sigma_3$': 4.005044211699847, '$\\sigma_4$': 7.860886653368249, '$\\sigma_5$': 1.4440694955487632, '$\\sigma_6$': 5.137093475763889, '$\\sigma_7$': 7.855418319751202, '$\\sigma_8$': 9.68119475129573, '$\\sigma_9$': 3.740584188529591, '$\\sigma_10$': 1.1215747898634054, '$\\sigma_11$': 5.245574885887011, '$\\sigma_12$': 8.02739579781354, '$\\sigma_13$': 3.620634858839847} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "D:\DeepLearning\Anaconda\envs\torch2\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\FabingDuan\AppData\Local\Temp\ipykernel_12232\3089027653.py", line 22, in objective
    for i, (images, labels) in enumerate(train_loader):
  File "D:\DeepLearning\Anaconda\envs\torch2\lib\site-packages\torch\utils\data\dataloader.py", li

KeyboardInterrupt: 